In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow
import keras
import nltk
import gensim

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
# from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout,SpatialDropout1D, Bidirectional,Flatten
from keras.models import Model
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn import model_selection, preprocessing
from sklearn.model_selection import train_test_split

from scipy.cluster.vq import whiten

from collections import Counter

from gensim.models import Word2Vec
from gensim.models import FastText

from sklearn.metrics import f1_score

In [3]:
import string
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, preprocessing
from sklearn.model_selection import train_test_split
import io
import matplotlib.pyplot as plt

from __future__ import print_function
from __future__ import division
import json
import numpy as np

from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense, Dropout, Flatten, Lambda, Embedding, LSTM,Bidirectional, concatenate, BatchNormalization 
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.initializers import RandomNormal
import tensorflow as tf
from sklearn.metrics import classification_report,confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

import pickle

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
import os
from pathlib import Path

path = Path(os.getcwd())
path = path/'gdrive'/'My Drive'
#news=path/'Thesis Data'/'full doc csv'
aa=path/'Thesis Data'/'Our dataset'
#aa2=path/'Thesis Data'/'AA dataset'
#char=path/'Thesis Data'/'char-ulm'
#wiki_data = path/'Thesis Data'/'wiki_data'/'Wiki'

In [6]:
def load_ag_data(trainfile,testfile):
  
  onehotencoder = preprocessing.OneHotEncoder(categories='auto')

  ## TRAIN
  df = pd.read_csv(trainfile)
  df = df.dropna()
  df = df.sample(frac=1).reset_index(drop=True)#shuffle
  x_train = df['text'].tolist()
  y_train = onehotencoder.fit_transform((df['label'].values).reshape(-1,1)).toarray()

  ## TEST
  df = pd.read_csv(testfile)
  df = df.dropna()
  df = df.sample(frac=1).reset_index(drop=True)#shuffle
  x_test = df['text'].tolist()
  y_test = onehotencoder.transform((df['label'].values).reshape(-1,1)).toarray()

  return (x_train, y_train), (x_test, y_test)

In [7]:
def create_vocab_set(data,MAX_SEQUENCE_LENGTH=750):
    vocab=Counter()
    for i,s in enumerate(data): # here use data's index and data
      vocab.update(s.split()) # split the 's'(data) and update on 'vocab',so vocab has all word in dataset

    with open(we,"rb") as input_file: # open 'rb' as a input file
      model = pickle.load(input_file) # python pickle is used to serialize and deserialize a python object structure
    word_vectors = model.wv 
    MAX_NB_WORDS = len(word_vectors.vocab) # 'MAX_NB_WORDS' is len of word_vectors vocabulary
    nb_words = min(MAX_NB_WORDS, len(word_vectors.vocab)) + 1
    word_index = {t[0]: i+1 for i,t in enumerate(vocab.most_common(MAX_NB_WORDS))}
    return vocab,word_index,nb_words,word_vectors,MAX_NB_WORDS

In [8]:
def encode_data(x, word_index, mxlen):
#   ret = [[word_index.get(t, 0) for t in sent.split() ] for sent in x]
  ret = []
  for sent in x: # loop over each sentence through dataset
    ls = [word_index.get(t, 0) for t in sent.split()] # each word in sentence take the 'word' and store it to 'ls',get method returns the value for the given key,if present in the dictionary.
    # padding
    if len(ls)>=mxlen: # if len(ls) is greater then 'mxlen' then we remove the extra area of 'ls' and update it
      ls = ls[:mxlen]
    else : # if len(ls) is smaller then mxlen then we apply zero padding of the extra area and add it to 'ls'
      ls=ls+[0]*(mxlen-len(ls))
    ret.append(ls) # after each sentence we append it to 'ret' list
  return np.array(ret) # return the 'ret' list as a np array

In [9]:
def form_we_matrix(word_index,nb_words,word_vectors,MAX_NB_WORDS,WV_DIM=300):
  # we initialize the matrix with random numbers
  wv_matrix = (np.random.rand(nb_words, WV_DIM) - 0.5) / 5.0

  for word, i in word_index.items():
      if i >= MAX_NB_WORDS:
          continue
      try:
          embedding_vector = word_vectors[word]
          wv_matrix[i] = embedding_vector
      except:
          pass
  return wv_matrix

In [10]:
def create_model(filter_kernels, dense_outputs, maxlen, vocab_size, nb_filter, cat_output, w, WV_DIM=300):
    
    initializer = RandomNormal(mean=0.0, stddev=0.05, seed=None)

    # Define what the input shape looks like
    inputs = Input(shape=(maxlen,), dtype='int64')

    # print(w)
    embedded = Embedding(input_dim=vocab_size, output_dim=WV_DIM , weights=[w])(inputs)

    # All the convolutional layers...
    conv = Convolution1D(filters=nb_filter[0], kernel_size=filter_kernels[0], kernel_initializer=initializer,
                         padding='valid', activation='relu',
                         input_shape=(maxlen, vocab_size))(embedded)
    conv = MaxPooling1D(pool_size=3)(conv)

    conv1 = Convolution1D(filters=nb_filter[1], kernel_size=filter_kernels[1], kernel_initializer=initializer,
                          padding='valid', activation='relu')(conv)
    conv1 = MaxPooling1D(pool_size=3)(conv1)

    lstm=Dropout(0.5)(LSTM(100)(conv1)) # after conv1 we use 100 'lstm' with dropout rate 0.5 
    z = Dropout(0.5)(Dense(dense_outputs, activation='relu')(lstm)) # applying 'relu' activation on top of 'lstm' layers

    # Output dense layer with softmax activation
    pred = Dense(cat_output, activation='softmax', name='output')(z)

    model = Model(inputs=inputs, outputs=pred)

    adam = Adam(lr=0.001, decay=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    return model


In [11]:
def CNN_LSTM(trainfile,testfile,aa,cat_output):
  np.random.seed(123)  # for reproducibility
  # Maximum length. Longer gets chopped. Shorter gets padded.
  maxlen = 750
  # Model params
  # Filters for conv layers
  nb_filter = [128,256]
  # Number of units in the dense layer
  dense_outputs = 512
  # Conv layer kernel size
  filter_kernels = [7, 3]
  # Compile/fit params
  batch_size = 128
  nb_epoch = 15

  print('Loading data...')
  (xt, yt), (x_test, y_test) = load_ag_data(trainfile,testfile)

  print('Creating vocab...')
  vocab,word_index,nb_words,word_vectors,MAX_NB_WORDS = create_vocab_set(xt)

  print('Form Word Embedding Matrix...')
  w = form_we_matrix(word_index,nb_words,word_vectors,MAX_NB_WORDS)
  print("word embedding shape = ",w.shape)
  
  print('Build model...')
  model = create_model(filter_kernels, dense_outputs, maxlen, nb_words, nb_filter, cat_output,w)
  
  print('Encode data...')
  xt = encode_data(xt, word_index, maxlen)
  x_test = encode_data(x_test, word_index, maxlen)

  model.summary()
  print('Fit model...')
  hist = model.fit(xt, yt, validation_split=0.2 , batch_size=batch_size, epochs=nb_epoch, shuffle=True)

  history = pd.DataFrame(hist.history)
  plt.figure(figsize=(12,12));
  plt.plot(history["loss"],label='loss');
  plt.plot(history["val_loss"], label='val-loss');
  plt.title("Loss with trained word vectors");
  plt.legend();
  # plt.show();
  plt.savefig(aa+'loss.eps')
  
  Y_pred = model.predict(x_test)
  y_pred = np.argmax(Y_pred, axis=1)
  y_test = np.argmax(y_test,axis=1)
  # print(y_pred)
  # print(y_test)
  acc = (y_test==y_pred).sum()/len(y_pred)
  f1 = f1_score(y_test, y_pred, average='macro')
  print("Accuracy=",acc," F1=",f1)
  cm=confusion_matrix(y_test, y_pred)
  print(cm)

  df_cm = pd.DataFrame(cm, range(cat_output),range(cat_output))
  plt.figure(figsize = (10,7))
  sn.set(font_scale=1.4)#for label size
  sn.heatmap(df_cm, annot=True,annot_kws={"size": 16},fmt='g')# font size
  plt.savefig(aa+'cm.eps')
  
  return model

In [12]:
model = CNN_LSTM(aa/'ulm_train.csv',aa/'ulm_test.csv','aa',6)

Loading data...
Creating vocab...


NameError: ignored